# Projeto ATD 2020/2021 
## Análise dos dados obtidos pelos acelerómetros de smartphones

#### Autores: 
 - André Carvalho, nº 2019216156
 - Joana Simões, nº 2019217013
 - Paulo Cortesão, nº 2019216517



In [3]:
%% Introdução : declaração de constantes a utilizar ao longo do trabalho

act= ["W","W\_UP", "W\_D", "SIT", "ST", "LAY", "ST\_SIT", "SIT\_ST","SIT\_LIE", "LIE\_SIT", "ST\_LIE", "LIE\_ST"]; %readlines('activity_labels.txt');
actFiles= ["W","W_UP", "W_D", "SIT", "ST", "LAY", "ST_SIT", "SIT_ST","SIT_LIE", "LIE_SIT", "ST_LIE", "LIE_ST"]; %readlines('activity_labels.txt');

d = ls('acc*.txt');
%disp(size(d,1))
%disp(d(1,:))
a = size(d,1);
lines=[1 23 46 66 86 107 127 148 168];


In [4]:
%%file readMatrix.m
function [mat, lines] = readMatrix(nameFile, nCol)
    file = fopen(nameFile, 'r');
    frmt = '%f';
    for k = 1:nCol-1
        frmt = strcat(frmt, ' %f');
    end
    mat = fscanf(file,frmt, [nCol Inf]);
    mat = mat';
end

Created file 'C:\Users\paulo\Desktop\UC2019 - LEI\2- Semestre\ATD\Projeto\readMatrix.m'.


In [5]:
%% 1. Importação dos dados a partir dos ficheiros de texto

labels=readMatrix('labels.txt', 5); 
labels= [labels; 0 0 0 0 0];
close all;

%disp(labels(1,:));

exp_user = [1,1; 2,1; 3,2; 4,2; 5,3; 6,3; 7,4; 8,4];

% disp(exp_user);


## 2. Representação dos dados

In [6]:
%%file representData.m
function []=representData(nameFile, exp_user, act, labels, line, fileIDs)
    vec = readMatrix(nameFile, 3);
    % ^ converter em funçºao esta última parte
    
    aid = size(vec,1);
    % the signals were captured at a frequency of 50 Hz -> T = 0.02s
    t = (0:0.02:((aid-1)*0.02))/60;
    figure()
    subplot(311)
    hold off;
    plot(t, vec(:,1), 'k')
    % y1 = ylim;
    ylim([-2.2,2.5]); 
    
    title('x axis')
    xlabel('t[min]')
    ylabel('acc')
    hold on
    subplot(312)
    hold off;
    plot(t, vec(:,2), 'k')
    % y1 = ylim;
    ylim([-2.2,2.5]); 
    title('y axis')
    xlabel('t[min]')
    ylabel('acc')
    hold on
    subplot(313)
    hold off;
    plot(t, vec(:,3),'k')
    y1 = ylim;
    ylim([-2.2,2.5]); 
    title('z axis')
    xlabel('t[min]')
    ylabel('acc')
    hold on
    nfigure=figure()
    pks_cell={{{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}}};
    freq_cell={{{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}} {{[]} {[]} {[]}}};
    % stft_samples(vec(:,3),256, 128, @hamming, 0.02);
    %TODO figure(); ->!FICA NOUTRO SÍTIO!
    % spectrogram(vec(:,3), 256, 128,[],50);
    lineBefore = line;
    begs=[];
    fin=0;
    while (~any(exp_user ~= labels(line,1:2)))
        figure(nfigure)
        activ=labels(line,3);     
        beg=labels(line,4);
        fin=labels(line,5);
        begs=[begs beg];
        % fins=[fins fin];
        med=(beg+fin) /2;
        for i = 1:3 
            subplot(3,1,i);
            plot(t(beg:fin), vec(beg:fin,i));
            k = -1.25;
            if mod(line,2)==0
                k=2.25;
            end
            text((med+beg)/2*0.02/60, k, act(activ));
            hold on;
        end
        [freq_cell, pks_cell] = dft_activity(vec(beg:fin,:), act(activ), activ, fileIDs(activ), freq_cell, pks_cell);
        % med=(beg+fin) /2;
    
        %t=annotation('textbox',[beg*0.02/60  fin*0.02/60],'String', act(activ));
        %sz = t.FontSize;
        %t.FontSize = 8;
        line=line+1;
    end

    % plot_freq_same_type(pks_cell, freq_cell,act);
    % plot_all_freq(pks_cell, freq_cell,act);
   

    %{for i =1:3
        %subplot(3,1,i);
        %xtickangle(45);
        %xticks(ticks.*0.02./60);
        %xticklabels(act(ticksl));
    %}


Created file 'C:\Users\paulo\Desktop\UC2019 - LEI\2- Semestre\ATD\Projeto\representData.m'.


**3.1.** Escolheu-se a atividade walking como ponto de partida. Calculou-se a DFT respetiva, de modo a identificar as frequências dominantes. 
Este processo foi levado a cabo tendo por base o seguinte código desenvolvido

In [ ]:
%%file dft_activity.m
function [freq_cell, pks_cell] = dft_activity(samples_activity, act_label, act, fileID, freq_cell, pks_cell)
    % exercício 3 - experiência
    ts = 0.02;
    threshold = 0.20;
    % file = fopen(nameFile, 'r');
    dim_names = ["x axis", "y axis", "z axis"];
    ham = hamming(numel(samples_activity(:, 1)));

    N = length(samples_activity(:, 1));
    nAxis = -1/2:1 / N:1/2 - 1 / N;

    if mod(N, 2) == 1
        nAxis = nAxis + 1 / (2 * N);
    end
    nAxis = nAxis / ts;
    %fileID = fopen(strcat(act_label, ".txt"), 'a');
    fprintf(fileID, 'New act\n');

    
    %k = figure();
    %k2 = figure();

    for i = 1:3
        %figure(k)
        %title(act_label);
        %subplot(3, 1, i)
        one_dir = samples_activity(:, i);
        dft_sample_one_dir = fftshift(fft(one_dir)) / numel(one_dir);
        dft_sample_one_dir(abs(dft_sample_one_dir) < 0.000001) = 0;
        dft_sample_one_dir_ham = fftshift(fft(one_dir .* ham)) / numel(one_dir); %!!!! perguntar ao prof: magnitude vs amplitude 
        dft_sample_one_dir_ham(abs(dft_sample_one_dir_ham) < 0.000001) = 0;
        %figure(k2)

        %plot((1:numel(one_dir)) * ts, one_dir);
        autoc = autocorr(one_dir, length(samples_activity) - 1);
        autoc = autoc(5:end);
        period = find(autoc >= 0.95 * max(autoc));

        if (size(period) >= 2)
            period = period(1);
        end

        tStep = period * ts;
        freqMin = 60 / tStep;
        %figure(k)

        %plot(nAxis, abs(dft_sample_one_dir));
        %hold on
        %plot(nAxis, abs(dft_sample_one_dir_ham));
        %xlabel('Frequência (Hz)');
        %ylabel(dim_names(i));

        mod_dft_sample_one_dir_ham = abs(dft_sample_one_dir_ham);
        mod_dft_sample_one_dir = abs(dft_sample_one_dir);
        min_mag = max(mod_dft_sample_one_dir_ham) * threshold;

        [pks, locs] = findpeaks(mod_dft_sample_one_dir, 'Minpeakheight', min_mag);
        %disp("Sizes:");
        %disp(size(locs));
        %disp(size(pks));
        f_relevant = nAxis(locs)';
        % disp(size(f_relevant));
        f_relevant(abs(f_relevant) < 0.001) = 0;
        %fprintf("Frequencies for %s", act_label)
        %disp([pks, f_relevant]);
        pks = pks(f_relevant >= 0);
        %disp("Peaks:")
        %disp(pks);
        f_relevant = f_relevant(f_relevant >= 0);

        if f_relevant(1) == 0 && length(pks) ~= 1
            pks = [pks(1); 2 * pks(2:end)];
        else
            pks= 2.*pks;
        end

        %{
        aidPks = pks(f_relevant > 0);
        aidPks = aidPks * 2;
        pks(f_relevant > 0) = aidPks;
        %}
        arrAid = sortrows([pks, f_relevant], 'descend');
        coordVal = i.*ones(size(pks));
        pks_cell{act}{i}{1} = [pks_cell{act}{i}{1}; pks(1:min([5, length(pks)]))];
        freq_cell{act}{i}{1} = [freq_cell{act}{i}{1}; f_relevant(1:min([5, length(f_relevant)]))];
        % actNo = ones(size(pks)).*act;
        % [f_relevant, pks, actNo]; -> argumento da função
        %sort(arrAid);
        
        % pks_cell{act}(i,:)= [ pks_cell{act}(i,:)  [arrAid(1:min[5, length(pks)],1 )]' ];
        % pks_freq{act}(i,:)= [ pks_freq{act}(i,:)  [arrAid(1:min[5, length(f_relevant)],2 )]' ];



        %disp(dim_names(i));
        %disp(f_relevant);
        %fprintf(fileID, '%s\n', dim_names(i));
        count = 0;

        for j = 1:numel(f_relevant)

            if count > 4
                break;
            end;
            fprintf(fileID, '%g %g\n', arrAid(j, 2), arrAid(j, 1));
            count = count + 1;
            end

            fprintf(fileID, 'Passos por minuto: %f\n', freqMin * 2);
            %{
            figure(k2)
            subplot(3, 1, i)
            plot(f_relevant, pks, 'o');
            xlabel('Frequência (Hz)');
            ylabel('Módulo da DFT');
            title(strcat(dim_names(i), strcat(" ", act_label)));
            %}
        end
        

    end
